### Model auswählen

In [ ]:
model_name = "mistralai/Mistral-7B-v0.3"

In [ ]:
model_name = "BioMistral/BioMistral-7B"

### Import Zone

In [ ]:
import torch
import pandas as pd
import re
from bs4 import BeautifulSoup
from datasets import Dataset
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_recall_fscore_support

from transformers import AutoTokenizer
from transformers import MistralForSequenceClassification, TrainingArguments, Trainer, BitsAndBytesConfig, DataCollatorWithPadding 


from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

import os
# Definieren Sie das Cache-Verzeichnis
cache_dir = '/media/ubuntu/5d2d9f9d-a02d-45ab-865f-3d789a0c70f0/download/'
os.environ['TRANSFORMERS_CACHE'] = cache_dir


### def Zone

In [ ]:
def clean_text(text):
    # HTML-Tags entfernen
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r"[\",\']","", text)  #  Anführungszeichen entfernen

    # 1. Mehrfache Anführungszeichen durch ein normales ' ersetzen
    text = re.sub(r"'{2,}", "'", text)

    # 2. HTML-Tags entfernen [1, 2, 3]
    # Sucht nach Mustern wie <tag>Inhalt</tag> und ersetzt sie durch einen leeren String.
    text = re.sub(r'<.*?>', '', text)

    # 3. URLs entfernen [1, 2, 3]
    # Sucht nach gängigen URL-Mustern (http/https, www.) und ersetzt sie durch einen leeren String.
    text = re.sub(r'http\S+|www\.\S+', '', text)

    # 4. E-Mail-IDs entfernen [3]
    # Sucht nach E-Mail-Mustern (Zeichenfolge@Zeichenfolge.Domain) und ersetzt sie durch einen leeren String.
    text = re.sub(r'\S*@\S*\s?', '', text)

    # 5. Zusätzliche Leerzeichen normalisieren [1, 4]
    # Teilt den Text nach Leerzeichen auf und fügt ihn mit einem einzigen Leerzeichen wieder zusammen.
    text = " ".join(text.split())

    text = re.sub(r"[\[,\]]","", text)  # Mehrfache Leerzeichen zu einem reduzieren
    

    return text

# Hilfsfunktion, um die Reduzierung der trainierbaren Parameter zu sehen
def print_trainable_parameters(model):
    """Prints the number of trainable parameters in the model."""
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || "
        f"trainable%: {100 * trainable_params / all_param:.2f}"
    )

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    
    # Berechnung des gewichteten F1-Scores
    f1 = f1_score(labels, preds, average='weighted')
    
    # Optional: Berechnung weiterer Metriken
    precision, recall, _, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0) # zero_division=0, um Warnungen zu vermeiden
    acc = accuracy_score(labels, preds)
    
    return {
        'f1': f1,
        'accuracy': acc,
        'precision': precision,
        'recall': recall
    }

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256) # max_length ggf. anpassen

### Dataset erstellen

In [ ]:
# --- 1. Klassen erweitern und Daten aus Pandas DataFrame laden ---

# Die Klassen wurden um "Umweltwissenschaft" und "Rest" erweitert.
id2label = {0: "Medizin", 1: "Ernährung", 2: "Landwirtschaft", 3: "Umweltwissenschaften", 4: "Rest"}
label2id = {key: value for value, key in id2label.items()}
NUM_LABELS = len(id2label)

# SIMULATION: Erstellen eines Beispiel-DataFrames.
# Ersetzen Sie diesen Block durch das Laden Ihrer eigenen Daten, z.B.:
# df = pd.read_csv('ihre_publikationen.csv')
path_train='../01_Daten/pkl/df_all_15k-2.pkl'
path_test='../01_Daten/pkl/df_val_5k-2.pkl'
df = pd.read_pickle(path_train)
df_test = pd.read_pickle(path_test)
df['text'] = df['title'].astype(str) + " - " + df['abstract'].astype(str)
df_test['text'] = df_test['title'].astype(str) + " - " + df_test['abstract'].astype(str)

# text cleanen
if "cased" in model_name:
    print("ohne lower...")
    df["text"] = df["text"].apply(clean_text)
    df_test['text_clean'] = df_test['text'].apply(clean_text)
else:
    df["text"] = df["text"].apply(clean_text).str.lower()
    df_test['text_clean'] = df_test['text'].apply(clean_text).str.lower()


df['text_clean'] = df['text'].apply(clean_text)
df_test['text_clean'] = df_test['text'].apply(clean_text)

# Umwandeln der Text-Labels (golden_record) in numerische IDs.
df['class'] = df['class'].str.replace(r'ErnÃ¤hrung', 'Ernährung', regex=True)
df_test['class'] = df_test['class'].str.replace(r'ErnÃ¤hrung', 'Ernährung', regex=True)
df['label_enc'] = df['class'].map(label2id)
df_test['label_enc'] = df_test['class'].map(label2id)

df['label_enc'] = df['label_enc'].astype(int)
df_test['label_enc'] = df_test['label_enc'].astype(int)


# Erstellen eines Hugging Face Datasets aus dem Pandas DataFrame
# Wir benötigen nur noch die Spalten 'text' und 'label'
final_df = df[['text_clean', 'label_enc']]
final_df_test = df_test[['text_clean', 'label_enc']]

# umbenennen der Spalten 
final_df=final_df.rename(columns={"label_enc":"labels"})
final_df=final_df.rename(columns={"text_clean":"text"})

final_df_test=final_df_test.rename(columns={"label_enc":"labels"})
final_df_test=final_df_test.rename(columns={"text_clean":"text"})

dataset = Dataset.from_pandas(final_df)

### Modell Laden und mit QLoRA "umhüllen"

In [ ]:
# --- 2. Modell und Tokenizer laden (mit 4-bit Quantisierung) ---

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)


model = MistralForSequenceClassification.from_pretrained(
    model_name,
    num_labels=NUM_LABELS,
    id2label=id2label,
    label2id=label2id,
    quantization_config=bnb_config,
    device_map="auto",
    cache_dir=cache_dir,
)
print("\nTrainierbare Parameter vor Anwendung von LoRA:")
print_trainable_parameters(model)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

# 1. Modell für das k-bit Training vorbereiten
model = prepare_model_for_kbit_training(model)

# 2. LoRA-Konfiguration erstellen
# Hier sagen wir PEFT, welche Schichten des Modells adaptiert werden sollen.
# Für Mistral sind das typischerweise die Aufmerksamkeits-Schichten.
lora_config = LoraConfig(
    r=16,                           # Rank der LoRA-Matrizen (üblicher Wert: 8, 16, 32)
    lora_alpha=32,                  # Alpha-Skalierungsfaktor
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS"             # Wichtig für Klassifizierungsaufgaben!
)

# 3. Das Basismodell mit der LoRA-Konfiguration "umwickeln"
model = get_peft_model(model, lora_config)
print("\nBasismodell wird mit der LoRA-Konfiguration \"umwickelt\"")

print("\nTrainierbare Parameter nach Anwendung von LoRA:")
print_trainable_parameters(model)

### tokeniesierung

In [ ]:
# --- 3. Daten für das Training vorbereiten ---
tokenized_dataset = dataset.map(tokenize_function, batched=True)

#Aufteilung in Trainings- und Testset für eine robustere Evaluierung
splits = tokenized_dataset.train_test_split(test_size=0.2)


Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

###  trainieren

In [ ]:
# --- 4. Training durchführen ---
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


training_args = TrainingArguments(
    output_dir=f"./mistral_classifier_results_v2/{model_name}",
    num_train_epochs=8,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_dir='./logs_lora/{model_name}',
    save_strategy="epoch",
    eval_strategy="epoch",
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=splits["train"],
    eval_dataset=splits["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("\nStarte das Fine-Tuning mit erweitertem Datensatz...")
trainer.train()
print("Training abgeschlossen!")

final_model_path = f"./mistral_classifier_final_v2/{model_name}"
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)
lora_config.save_pretrained(final_model_path)

print(f"Finales Modell wurde unter {final_model_path} gespeichert.")



No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.



Starte das Fine-Tuning mit erweitertem Datensatz...


/media/ubuntu/5d2d9f9d-a02d-45ab-865f-3d789a0c70f0/BA/Novak/.venv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,F1,Accuracy,Precision,Recall
1,No log,0.842040,0.686106,0.689333,0.704341,0.689333
2,No log,0.890496,0.652152,0.655667,0.702051,0.655667
3,No log,0.887447,0.709602,0.707667,0.713755,0.707667
4,No log,1.183387,0.685023,0.683333,0.697787,0.683333
5,No log,1.700010,0.683649,0.684000,0.691688,0.684000
6,0.742400,2.146274,0.692248,0.692333,0.694559,0.692333
7,0.742400,2.746061,0.696087,0.697333,0.696978,0.697333
8,0.742400,2.884153,0.695430,0.695333,0.698191,0.695333


/media/ubuntu/5d2d9f9d-a02d-45ab-865f-3d789a0c70f0/BA/Novak/.venv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/media/ubuntu/5d2d9f9d-a02d-45ab-865f-3d789a0c70f0/BA/Novak/.venv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences

Training abgeschlossen!
Finales Modell wurde unter ./mistral_classifier_final_v2 gespeichert.
